In [1]:
!python3 -m pip install --upgrade pip
!python3 -m pip install accelerate -U


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from tqdm import tqdm


/Users/ajaypokharel/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [12]:
data = pd.read_csv('assets/transcripts_with_notes.csv')
lectures = data['lecture'].to_list()
summaries = data['summarization'].to_list()
print(type(lectures[0]))

<class 'str'>


In [13]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-base')

train_encodings = tokenizer(lectures, summaries, truncation=True, padding='max_length', max_length=512)
print(train_encodings)


{'input_ids': [[0, 1121, 42, 569, 6, 38, 437, 164, 7, 304, 1617, 101, 38695, 6, 5788, 6, 8, 4864, 7, 9115, 7, 5561, 9, 86, 11, 5, 1050, 50, 11, 5, 1198, 12, 19003, 375, 4, 178, 99, 38, 236, 7, 14246, 235, 31, 5, 120, 12, 2977, 6, 142, 13542, 42, 16, 402, 14, 18, 10985, 162, 11, 5, 375, 6, 16, 14, 37325, 40, 9115, 7, 38695, 6, 5788, 6, 8, 4864, 11, 5, 1050, 375, 6, 8, 51, 214, 2333, 5056, 7, 5788, 9, 7281, 9, 1583, 9, 107, 50, 1583, 9, 107, 4, 125, 209, 32, 430, 38695, 6, 5788, 6, 8, 4864, 87, 5, 1980, 14, 5473, 10974, 74, 9115, 7, 77, 51, 214, 1686, 59, 30694, 86, 4, 96, 30694, 86, 6, 3567, 839, 484, 6317, 2535, 9, 107, 4, 18365, 29, 8, 4864, 1266, 2535, 9, 107, 4, 520, 37325, 6, 77, 52, 214, 1158, 5, 1050, 375, 6, 51, 214, 95, 3489, 1686, 59, 251, 5561, 9, 1050, 86, 6, 53, 45, 11, 5, 2535, 9, 107, 6, 2333, 11, 5, 1583, 50, 5, 158, 6, 151, 29, 9, 107, 4, 407, 99, 38, 236, 7, 109, 19, 14, 66, 9, 5, 169, 16, 1067, 59, 99, 34, 1102, 11, 5, 13258, 1050, 375, 50, 5, 13258, 1198, 12, 19003, 

In [23]:
model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-base')

training_args = TrainingArguments(
    output_dir='./bart-summarization',
    num_train_epochs=2,
    per_device_train_batch_size=16,
    warmup_steps=0,
    save_steps=5,
    save_total_limit=2
)


In [24]:
print(training_args)

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
gradient_checkpointing_kwargs=None,
greater_is_better=None,
group_by_length=False,
half_precision_backend=au

In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encodings
)

trainer.train()


  0%|          | 0/2 [00:00<?, ?it/s]

ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: input_ids,attention_mask,decoder_input_ids,decoder_attention_mask,head_mask,decoder_head_mask,cross_attn_head_mask,encoder_outputs,past_key_values,inputs_embeds,decoder_inputs_embeds,labels,use_cache,output_attentions,output_hidden_states,return_dict,label,label_ids,labels.